In [1]:
import os
import json
import requests 
import pandas as pd
import numpy as np
from datetime import datetime
from config import riot_api

In [2]:
now = datetime.now() # Get current datetime
date = now.date().strftime("%B %d, %Y")
ogtime = now.time().strftime("%H:%M:%S")

# region code is [{BR1:"Brazil",EUN1:"EU Nordic & East",EUW1:"EU West",JP1:"Japan",KR:"Korea",LA1:"Latin America North" 
# LA2:"Latin America South,NA1:"North America,OC1:"Oceania",PBE1:"Public Beta Environment,RU:"Russia",TR1:Turkey}]

region_code = input("Enter Your Region: ").upper() # Ask for region code

if region_code in ["NA1", "BR1", "LA1", "LA2", "OC1"]: # Get region
    region = "americas"
elif region_code in ["KR", "JP1"]:
    region = "asia"
else:
    region = "europe"

try:
    os.makedirs(f"Analysis/{region_code}/{date}") # Make new  directory
except:
    print("Today's directory has been created already")
    
# Free Champions Rotation

url = f"https://{region_code}.api.riotgames.com/lol/platform/v3/champion-rotations?api_key=" + riot_api
champ_list = requests.get(url).json()
with open(f'Analysis/{region_code}/{date}/champ_rotation.json', 'w') as file: 
    file.write(json.dumps(champ_list))

# Live Games

url = f"https://{region_code}.api.riotgames.com/lol/spectator/v4/featured-games?api_key=" + riot_api
live_games = requests.get(url)
statcode = live_games.status_code

if statcode == 200:
    live_games_status = "Region availible"
else:
    live_games_status ="Region unavailible"

print(f"{live_games_status}\n{live_games.status_code}\n{champ_list}") 


Enter Your Region: euw1
Today's directory has been created already
Region availible
200
{'freeChampionIds': [17, 43, 56, 62, 67, 79, 85, 90, 133, 145, 147, 157, 201, 203, 245, 518], 'freeChampionIdsForNewPlayers': [222, 254, 427, 82, 131, 147, 54, 17, 18, 37], 'maxNewPlayerLevel': 10}


In [3]:
# Save Live Game Data

live_games = live_games.json()
live_games_df = pd.DataFrame(live_games['gameList'])
live_games_df.to_csv(f'Analysis/{region_code}/{date}/live_game_data({ogtime}).csv')
    
live_games_df

,gameId,mapId,gameMode,gameType,gameQueueConfigId,participants,observers,platformId,bannedChampions,gameStartTime,gameLength
0,5860943084,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 4, 'spell2Id': 21...",{'encryptionKey': 'Z76Bg+WS8qHwnk+BU/338hshkyl...,EUW1,[],1651960137714,120
1,5860942534,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 4, 'spell2Id': 3,...",{'encryptionKey': '19jEJv4hqlcwfAnEJD1B87MIQ90...,EUW1,[],1651959946938,310
2,5860972350,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 7, 'spell2Id': 4,...",{'encryptionKey': 'uxlTPQTiD88FgvftbCWH5tnRR8X...,EUW1,[],1651960178708,79
3,5860981634,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 4, 'spell2Id': 32...",{'encryptionKey': 'ZtPuEkSYLcV9feoZwk7Pt3Bb+S9...,EUW1,[],1651959963660,294
4,5860923153,11,CLASSIC,MATCHED_GAME,420,"[{'teamId': 100, 'spell1Id': 11, 'spell2Id': 3...",{'encryptionKey': '3/QvjDgdC7xVUJO/AF613gmMWWy...,EUW1,"[{'championId': 38, 'teamId': 100, 'pickTurn':...",1651960160754,97


In [4]:
name = input("Enter Your Summoner Name: ") # And Username
try:
    os.makedirs(f"Analysis/{region_code}/{date}/{name}") # Make new  directory
except:
    print("Summoners directory has already been created")

Enter Your Summoner Name: curling captain
Summoners directory has already been created


In [5]:
# Summoners Data 

def Summoner (region_code, name):    
    
    url = f"https://{region_code}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key=" + riot_api
    summoner_info_1 = requests.get(url).json()
    iden, puuid = summoner_info_1["id"], summoner_info_1["puuid"]
    
    Ranked_info_url = f"https://{region_code}.api.riotgames.com/lol/league/v4/entries/by-summoner/{iden}?api_key=" + riot_api
    Champ_Pool_Mastery_url = f"https://{region_code}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{iden}?api_key=" + riot_api

    Ranked_info = requests.get(Ranked_info_url).json()    
    Champ_Pool_Mastery = requests.get(Champ_Pool_Mastery_url).json()
    if live_games_status == "Region availible":
        url = f"https://{region_code}.api.riotgames.com/lol/spectator/v4/active-games/by-summoner/{iden}?api_key=" + riot_api
        in_game_info =  requests.get(url)
        if in_game_info.status_code == "200":
            in_game_info =  in_game_info.json()
            print("In Live Game")
        else:
            in_game_info = np.nan
            print("No Live Data")
    else:
        in_game_info = np.nan
        
    return [summoner_info_1, Ranked_info, Champ_Pool_Mastery, in_game_info]

summoner_info = Summoner(region_code, name)
Summoner_info, Ranked_info, Champ_Pool_Mastery = [summoner_info[0]], summoner_info[1], summoner_info[2]



No Live Data


In [6]:
# Get Summoner bio

Summoner_bio_df = pd.DataFrame(Summoner_info)

Summoner_bio_df.to_csv(f'Analysis/{region_code}/{date}/{name}/data({ogtime}).csv', index=False)

Summoner_bio_df.head()

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,N9hJ-oCIRecZEJplZQT7yHmkZitN7Itr6bv6eBM2vBOlkFA,N8Gm-AkHSS7KV0nYsCNfQYNLEutkbQnKogm-kW6PEle5WQ,-F5baFL9oke6dFlIWHCAlkcDteYCen3ee2UvRZLNWJWTcO...,Curling Captain,4030,1651831803000,263


In [7]:
# Get Ranked Info, if exists

if not Ranked_info:
    print("Unranked this Season")
    Ranked_info_df = pd.DataFrame()    

else:
    Ranked_info_df = pd.DataFrame(Ranked_info)
    Ranked_info_df.to_csv(f'Analysis/{region_code}/{date}/{name}/ranked_data_({ogtime}).csv', index=False)


Ranked_info_df.head()

,queueType,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,leagueId,tier,rank
0,RANKED_TFT_PAIRS,N9hJ-oCIRecZEJplZQT7yHmkZitN7Itr6bv6eBM2vBOlkFA,Curling Captain,0,5,16,False,False,False,False,NaN,NaN,NaN
1,RANKED_SOLO_5x5,N9hJ-oCIRecZEJplZQT7yHmkZitN7Itr6bv6eBM2vBOlkFA,Curling Captain,558,423,403,False,False,True,False,f6134c21-afc2-3c72-8b43-5259ec512b58,MASTER,I
2,RANKED_FLEX_SR,N9hJ-oCIRecZEJplZQT7yHmkZitN7Itr6bv6eBM2vBOlkFA,Curling Captain,84,22,4,False,False,False,True,945582fc-0c8f-422a-93f7-8bdbf50a77da,GOLD,II


In [8]:
# Get Champion Pool Info

print(f"{region_code}, {region}, {name}")
Champ_Pool_Mastery_df = pd.DataFrame(Champ_Pool_Mastery)
Champ_Pool_Mastery_df.to_csv(f'Analysis/{region_code}/{date}/{name}/champ_pool_({ogtime}).csv', index=False)

Champ_Pool_Mastery_df.head()

EUW1, europe, curling captain


,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,64,5,316628,1651770703000,295028,0,True,2,N9hJ-oCIRecZEJplZQT7yHmkZitN7Itr6bv6eBM2vBOlkFA
1,76,5,210466,1651690147000,188866,0,True,2,N9hJ-oCIRecZEJplZQT7yHmkZitN7Itr6bv6eBM2vBOlkFA
2,60,5,207134,1651666395000,185534,0,True,2,N9hJ-oCIRecZEJplZQT7yHmkZitN7Itr6bv6eBM2vBOlkFA
3,421,5,80536,1651752165000,58936,0,True,2,N9hJ-oCIRecZEJplZQT7yHmkZitN7Itr6bv6eBM2vBOlkFA
4,25,5,60355,1651671290000,38755,0,True,2,N9hJ-oCIRecZEJplZQT7yHmkZitN7Itr6bv6eBM2vBOlkFA


In [9]:
# Get Recent Game info 

def Games(puuid, region):
    
    last_20_id_url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key=" + riot_api
    last_20_id = requests.get(last_20_id_url).json()
    
    match_data = [(requests.get(f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match}?api_key=" + riot_api).json()) 
                       for match in last_20_id] 
    time_line = [(requests.get(f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match}/timeline?api_key=" + riot_api).json()) 
                       for match in last_20_id]
    
    return [last_20_id, match_data, time_line]

puuid = Summoner_bio_df["puuid"][0]
Last_20_Games_Metadata = Games(puuid, region)
Last_20_Games_Metadata_df = pd.DataFrame(Last_20_Games_Metadata)
try:
    os.makedirs(f"Analysis/{region_code}/{date}/{name}/Last_20_({ogtime})") # Make new  directory
except: 
    print("Folder has been created")

for j in range(len(Last_20_Games_Metadata_df.columns)): # for each match store its data and live data in new json elements
    game_id = Last_20_Games_Metadata[0][j]
    with open(f'Analysis/{region_code}/{date}/{name}/Last_20_({ogtime})/{game_id}.json', 'w') as file: 
            file.write(json.dumps(Last_20_Games_Metadata[1][j]))
    with open(f'Analysis/{region_code}/{date}/{name}/Last_20_({ogtime})/timeline_{game_id}.json', 'w') as file:
            file.write(json.dumps(Last_20_Games_Metadata[2][j]))

Last_20_Games_Metadata_df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,EUW1_5858260985,EUW1_5858217262,EUW1_5857472612,EUW1_5857406464,EUW1_5857339420,EUW1_5857304837,EUW1_5857227945,EUW1_5857223661,EUW1_5857140191,EUW1_5857105948,EUW1_5857102812,EUW1_5856112216,EUW1_5855997570,EUW1_5856061605,EUW1_5855918226,EUW1_5855921467,EUW1_5855876088,EUW1_5855800850,EUW1_5855746057,EUW1_5855652361
1,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
2,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."


In [10]:
# Write inputs to seperate file

with open(f'Analysis/user_input.py', 'w') as file:
    file.write("region_code ="+ f' "{region_code}"\n')
    file.write("region =" + f' "{region}"')
   